# Infer ***lysosome*** -  4️⃣

--------------

## OVERVIEW
In notesbooks 4-9, we will go how each organelle is segmented. These segmentation workflows are completely independent of each other and can be run in any order. Additionally, the entire image will be processed in this notebook, irrespective of the single cells identified in notebooks 1-3. The cell mask will be applied to the organelle segmentation outputs before quantification (as outlined in [10_regionprops](./10_regionprops.ipynb)). 

This notebook goes through the workflow steps to segment the ***lysosomes*** -- one of the six organelles of interest.

## OBJECTIVE: 
### ✅ Infer sub-cellular component ***lysosome***
Segment the ***lysosomes*** from a single channel (lysosomal membrane marker). This workflow was optimized for images of fluorescently tagged Lamp1, a membrane protein. When lysosomes are larger in size, this causes a hollow sphere structure that requires different segmentation compared to smaller lysosomes that appear filled in.


## IMPORTS

In [2]:
# top level imports
from pathlib import Path
import os, sys
from typing import Optional

import numpy as np

from aicssegmentation.core.seg_dot import dot_2d_slice_by_slice_wrapper
from aicssegmentation.core.vessel import filament_2d_wrapper
from aicssegmentation.core.utils import topology_preserving_thinning
from skimage.measure import label
import skimage
import scipy

import napari

### import local python functions in ../infer_subc
sys.path.append(os.path.abspath((os.path.join(os.getcwd(), '..'))))


from infer_subc.core.file_io import (read_czi_image,
                                                                    read_ome_image,
                                                                    export_inferred_organelle,
                                                                    import_inferred_organelle,
                                                                    list_image_files)
from infer_subc.core.img import * #dot_filter_3, filament_filter_3
from infer_subc.organelles import fixed_infer_cellmask_fromcomposite, fixed_infer_nuclei_fromlabel, get_cytoplasm, get_nuclei, get_cellmask
# NOTE:  these "constants" are only accurate for the testing MCZ dataset
from infer_subc.constants import (TEST_IMG_N,
                                                                    NUC_CH ,
                                                                    LYSO_CH ,
                                                                    MITO_CH ,
                                                                    GOLGI_CH ,
                                                                    PEROX_CH ,
                                                                    ER_CH ,
                                                                    LD_CH ,
                                                                    RESIDUAL_CH )              

%load_ext autoreload
%autoreload 2


## Get and load Image for processing

In [3]:
test_img_n = TEST_IMG_N

data_root_path = Path(os.path.expanduser("~")) /  "Documents/Python Scripts/Infer-subc-2D"

in_data_path = data_root_path / "raw"
im_type = ".czi"

img_file_list = list_image_files(in_data_path,im_type)
test_img_name = img_file_list[test_img_n]

out_data_path = data_root_path / "out"
if not Path.exists(out_data_path):
    Path.mkdir(out_data_path)
    print(f"making {out_data_path}")

In [4]:
img_data,meta_dict = read_czi_image(test_img_name)

channel_names = meta_dict['name']
img = meta_dict['metadata']['aicsimage']
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']

c:\Users\Shannon\anaconda3\envs\infer-subc\lib\site-packages\ome_types\_convenience.py:112: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


---------------------

## infer ***lysosomes***

## summary of steps

➡️ INPUT
- select single channel containing the lysosome marker (channel number = user input)

PRE-PROCESSING
- rescale image intensities: 
    - min=0, max=1
- smooth image:
    - median filter (media size = user input)
    - gaussian filter (sigma = user input)

CORE PROCESSING
- apply "dot" thresholding method (for small round objects) from the Allen Cell [aicssegmentation](https://github.com/AllenCell/aics-segmentation) package (size scale and threshold cutoff = user input)
- apply "filament"/"vessel" thresholding method (for tubular objects) from the Allen Cell [aicssegmentation](https://github.com/AllenCell/aics-segmentation) package (size scale and threshold cutoff = user input)
- combine the two segmentations with logical *OR*

POST-PROCESSING
  - fill holes (hole size = user input)
  - remove small objects (object size = user input)

OUTPUT ➡️
- label unique lysosome objects based on connectivity
- save labeled ***lysosomes*** (lyso, LS) as unsigned integer 16-bit tif files


> Generally, we followed the Allen Cell Segmenter procedure for lysosome segmentation from the [Lamp1](https://www.allencell.org/cell-observations/category/lamp1) marker. Sourced from: this notebook [example](https://github.com/AllenCell/aics-segmentation/blob/main/lookup_table_demo/playground_lamp1.ipynb)



## EXTRACT prototype

In [5]:
###################
# INPUT
###################
raw_lyso = select_channel_from_raw(img_data, LYSO_CH)

## PRE-PROCESSING prototype


In [6]:
###################
# PRE_PROCESSING
###################
med_filter_size = 3   
gaussian_smoothing_sigma = 1.34

struct_img =  scale_and_smooth(raw_lyso,
                               median_size = med_filter_size, 
                               gauss_sigma = gaussian_smoothing_sigma)

## CORE PROCESSING prototype

In [7]:
###################
# CORE_PROCESSING
###################
## PARAMETERS for this step ##
# s_param = [[5,0.09], [2.5,0.07], [1,0.01]]
dot_method = "3D"

# apply the 2D or 3D versions of the AICSsegmentation dot filter with multiple scales
bw_dot_test = dot_filter_3(struct_img, 5, 0.09, 2.5, 0.07, 1, 0.01, dot_method)


################################
## PARAMETERS for this step ##
# f_param = [[1, 0.15]]
fil_method = "3D"

# apply the 2D or 3D versions of the AICSsegmentation filament filter with multiple scales
bw_filament_test = filament_filter_3(struct_img, 1, 0.15, 0,0,0,0, fil_method)


# combine the two segmentations together
bw_test = np.logical_or(bw_dot_test, bw_filament_test)


## POST-PROCESSING

In [10]:
###################
# POST_PROCESSING
###################
hole_min_width = 0
hole_max_width = 25

small_object_width = 1

fill_filter_method = "3D"

cleaned_img2 = fill_and_filter_linear_size(bw_test, 
                                           hole_min=hole_min_width, 
                                           hole_max=hole_max_width, 
                                           min_size=small_object_width,
                                           method=fill_filter_method)


[autoreload of skimage.util._montage failed: Traceback (most recent call last):
  File "c:\Users\Shannon\anaconda3\envs\infer-subc\lib\site-packages\IPython\extensions\autoreload.py", line 273, in check
    superreload(m, reload, self.old_objects)
  File "c:\Users\Shannon\anaconda3\envs\infer-subc\lib\site-packages\IPython\extensions\autoreload.py", line 496, in superreload
    update_generic(old_obj, new_obj)
  File "c:\Users\Shannon\anaconda3\envs\infer-subc\lib\site-packages\IPython\extensions\autoreload.py", line 393, in update_generic
    update(a, b)
  File "c:\Users\Shannon\anaconda3\envs\infer-subc\lib\site-packages\IPython\extensions\autoreload.py", line 305, in update_function
    setattr(old, name, getattr(new, name))
ValueError: montage() requires a code object with 2 free vars, not 0
]
[autoreload of skimage.util.noise failed: Traceback (most recent call last):
  File "c:\Users\Shannon\anaconda3\envs\infer-subc\lib\site-packages\IPython\extensions\autoreload.py", line 273,

In [11]:
viewer = napari.Viewer()

In [12]:
viewer.add_image(img_data)

viewer.add_image(struct_img)

viewer.add_image(cleaned_img2, colormap="cyan", opacity=0.3)

<Image layer 'cleaned_img2' at 0x19b9d19e440>

#### Labeling TEST

In [13]:
lyso_basic_labels = label(cleaned_img2)
viewer.add_image(lyso_basic_labels)

unique_IDs = np.unique(lyso_basic_labels)

[autoreload of skimage.measure._colocalization failed: Traceback (most recent call last):
  File "c:\Users\Shannon\anaconda3\envs\infer-subc\lib\site-packages\IPython\extensions\autoreload.py", line 273, in check
    superreload(m, reload, self.old_objects)
  File "c:\Users\Shannon\anaconda3\envs\infer-subc\lib\site-packages\IPython\extensions\autoreload.py", line 471, in superreload
    module = reload(module)
  File "c:\Users\Shannon\anaconda3\envs\infer-subc\lib\importlib\__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 619, in _exec
  File "<frozen importlib._bootstrap_external>", line 883, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "c:\Users\Shannon\anaconda3\envs\infer-subc\lib\site-packages\skimage\measure\_colocalization.py", line 4, in <module>
    from .._shared.utils import check_shape_equality, as_binary_ndarray
ImportError: cannot import name 'as_binar

In [74]:
obj_two = (lyso_basic_labels==2)*2
obj_three = (lyso_basic_labels==3)*3
obj_four = (lyso_basic_labels==4)*4
obj_five = (lyso_basic_labels==5)*5
np.unique(obj_two), np.unique(obj_three), np.unique(obj_four), np.unique(obj_five)

(array([0, 2]), array([0, 3]), array([0, 4]), array([0, 5]))

In [75]:
new_array_even = np.add(obj_four, obj_two)
new_array_odd = np.add(obj_three, obj_five)

np.unique(new_array_even), np.unique(new_array_odd)

(array([0, 2, 4]), array([0, 3, 5]))

In [87]:
combined_array = np.add(new_array_even, new_array_odd)
np.unique(combined_array)

array([0, 2, 3, 4, 5])

In [93]:
renumber_array = skimage.segmentation.relabel_sequential(combined_array, 6)
np.unique(renumber_array[0])

array([0, 6, 7, 8, 9])

In [70]:
np.unique(new_array)

array([0, 2, 4])

In [31]:
lst = np.unique(four_eroded_skimage).astype(int)
np.max(lst)

1

<Image layer 'four_eroded_skimage_2' at 0x19ba73c50c0>

In [18]:
np.shape(four_eroded_skimage), np.shape(four_eroded_skimage_2)

((17, 704, 704), (17, 704, 704))

In [22]:
viewer.add_image(~obj_four)

<Image layer 'Image' at 0x19ba7225840>

In [27]:
whats_left_TEST = skimage.segmentation.watershed(~obj_four, markers=four_eroded_skimage_2, mask=obj_four)
the_difference = np.logical_xor(obj_four, whats_left_TEST)
viewer.add_image(whats_left_TEST)
viewer.add_image(the_difference)

<Image layer 'the_difference' at 0x19c0d4940d0>

In [44]:
four_eroded_skimage = skimage.morphology.isotropic_erosion(obj_four, 0.1, spacing=scale)
four_eroded_skimage_2 = skimage.morphology.isotropic_erosion(four_eroded_skimage, 0.1, spacing=scale)
four_eroded_skimage_3 = skimage.morphology.isotropic_erosion(four_eroded_skimage_2, 0.1, spacing=scale)
four_eroded_skimage_4 = skimage.morphology.isotropic_erosion(four_eroded_skimage_3, 0.1, spacing=scale)
four_eroded_skimage_5 = skimage.morphology.isotropic_erosion(four_eroded_skimage_4, 0.1, spacing=scale)
four_eroded_skimage_6 = skimage.morphology.isotropic_erosion(four_eroded_skimage_5, 0.1, spacing=scale)
four_eroded_skimage_7 = skimage.morphology.isotropic_erosion(four_eroded_skimage_6, 0.1, spacing=scale)
four_eroded_skimage_8 = skimage.morphology.isotropic_erosion(four_eroded_skimage_7, 0.1, spacing=scale)
four_eroded_skimage_9 = skimage.morphology.isotropic_erosion(four_eroded_skimage_8, 0.1, spacing=scale)
four_eroded_skimage_10 = skimage.morphology.isotropic_erosion(four_eroded_skimage_9, 0.1, spacing=scale)
four_eroded_skimage_11 = skimage.morphology.isotropic_erosion(four_eroded_skimage_10, 0.1, spacing=scale)
four_eroded_skimage_12 = skimage.morphology.isotropic_erosion(four_eroded_skimage_11, 0.1, spacing=scale)
four_eroded_skimage_13 = skimage.morphology.isotropic_erosion(four_eroded_skimage_12, 0.1, spacing=scale)
four_eroded_skimage_14 = skimage.morphology.isotropic_erosion(four_eroded_skimage_13, 0.1, spacing=scale)
four_eroded_skimage_15 = skimage.morphology.isotropic_erosion(four_eroded_skimage_14, 0.1, spacing=scale)
four_eroded_skimage_16 = skimage.morphology.isotropic_erosion(four_eroded_skimage_15, 0.1, spacing=scale)




<Image layer 'four_eroded_skimage_16' at 0x19bb2c45bd0>

In [49]:
viewer.add_image(four_eroded_skimage_13)

<Image layer 'four_eroded_skimage_13' at 0x19ba69235b0>

In [57]:
whats_left_TEST = skimage.segmentation.watershed(~obj_four, markers=four_eroded_skimage, mask=obj_four)
whats_left_TEST_2 = skimage.segmentation.watershed(~four_eroded_skimage, markers=four_eroded_skimage_2, mask=four_eroded_skimage)
whats_left_TEST_3 = skimage.segmentation.watershed(~four_eroded_skimage_2, markers=four_eroded_skimage_3, mask=four_eroded_skimage_2)
whats_left_TEST_4 = skimage.segmentation.watershed(~four_eroded_skimage_3, markers=four_eroded_skimage_4, mask=four_eroded_skimage_3)
whats_left_TEST_5 = skimage.segmentation.watershed(~four_eroded_skimage_4, markers=four_eroded_skimage_5, mask=four_eroded_skimage_4)
whats_left_TEST_6 = skimage.segmentation.watershed(~four_eroded_skimage_5, markers=four_eroded_skimage_6, mask=four_eroded_skimage_5)
whats_left_TEST_7 = skimage.segmentation.watershed(~four_eroded_skimage_6, markers=four_eroded_skimage_2, mask=four_eroded_skimage_6)
whats_left_TEST_8 = skimage.segmentation.watershed(~four_eroded_skimage_7, markers=four_eroded_skimage_3, mask=four_eroded_skimage_7)
whats_left_TEST_9 = skimage.segmentation.watershed(~four_eroded_skimage_8, markers=four_eroded_skimage_4, mask=four_eroded_skimage_8)
whats_left_TEST_10 = skimage.segmentation.watershed(~four_eroded_skimage_9, markers=four_eroded_skimage_5, mask=four_eroded_skimage_9)
whats_left_TEST_11 = skimage.segmentation.watershed(~four_eroded_skimage_10, markers=four_eroded_skimage_6, mask=four_eroded_skimage_10)
whats_left_TEST_12 = skimage.segmentation.watershed(~four_eroded_skimage_11, markers=four_eroded_skimage_2, mask=four_eroded_skimage_11)
whats_left_TEST_13 = skimage.segmentation.watershed(~four_eroded_skimage_12, markers=four_eroded_skimage_3, mask=four_eroded_skimage_12)
whats_left_TEST_14 = skimage.segmentation.watershed(~four_eroded_skimage_13, markers=four_eroded_skimage_4, mask=four_eroded_skimage_13)
whats_left_TEST_15 = skimage.segmentation.watershed(~four_eroded_skimage_14, markers=four_eroded_skimage_5, mask=four_eroded_skimage_14)
whats_left_TEST_16 = skimage.segmentation.watershed(~four_eroded_skimage_15, markers=four_eroded_skimage_6, mask=four_eroded_skimage_15)

the_difference_ = np.logical_xor(obj_four, whats_left_TEST)
the_difference_2 = np.logical_xor(four_eroded_skimage, whats_left_TEST_2)
the_difference_3 = np.logical_xor(four_eroded_skimage_2, whats_left_TEST_3)
the_difference_4 = np.logical_xor(four_eroded_skimage_3, whats_left_TEST_4)
the_difference_5 = np.logical_xor(four_eroded_skimage_4, whats_left_TEST_5)
the_difference_6 = np.logical_xor(four_eroded_skimage_5, whats_left_TEST_6)
the_difference_7 = np.logical_xor(four_eroded_skimage_6, whats_left_TEST_7)
the_difference_8 = np.logical_xor(four_eroded_skimage_7, whats_left_TEST_8)
the_difference_9 = np.logical_xor(four_eroded_skimage_8, whats_left_TEST_9)
the_difference_10 = np.logical_xor(four_eroded_skimage_9, whats_left_TEST_10)
the_difference_11 = np.logical_xor(four_eroded_skimage_10, whats_left_TEST_11)
the_difference_12 = np.logical_xor(four_eroded_skimage_11, whats_left_TEST_12)
the_difference_13 = np.logical_xor(four_eroded_skimage_12, whats_left_TEST_13)
the_difference_14 = np.logical_xor(four_eroded_skimage_13, whats_left_TEST_14)
the_difference_15 = np.logical_xor(four_eroded_skimage_14, whats_left_TEST_15)
the_difference_16 = np.logical_xor(four_eroded_skimage_15, whats_left_TEST_16)

In [55]:
viewer.add_image(whats_left_TEST_2)

<Image layer 'whats_left_TEST_2' at 0x19c12d18f10>

In [56]:
viewer.add_image(the_difference_2)

<Image layer 'the_difference_2' at 0x19c0d34bf70>

In [ ]:
viewer.add_image(four_eroded_skimage, opacity=0.3)
viewer.add_image(four_eroded_skimage_2, opacity=0.3)
viewer.add_image(four_eroded_skimage_3, opacity=0.3)
viewer.add_image(four_eroded_skimage_4, opacity=0.3)
viewer.add_image(four_eroded_skimage_5, opacity=0.3)
viewer.add_image(four_eroded_skimage_6, opacity=0.3)
viewer.add_image(four_eroded_skimage_7, opacity=0.3)
viewer.add_image(four_eroded_skimage_8, opacity=0.3)
viewer.add_image(four_eroded_skimage_9, opacity=0.3)
viewer.add_image(four_eroded_skimage_10, opacity=0.3)
viewer.add_image(four_eroded_skimage_11, opacity=0.3)
viewer.add_image(four_eroded_skimage_12, opacity=0.3)
viewer.add_image(four_eroded_skimage_13, opacity=0.3)
viewer.add_image(four_eroded_skimage_14, opacity=0.3)
viewer.add_image(four_eroded_skimage_15, opacity=0.3)
viewer.add_image(four_eroded_skimage_16, opacity=0.3)

In [48]:
viewer.add_image(whats_left_TEST, opacity=0.3)

<Image layer 'whats_left_TEST' at 0x19bb578d5d0>

In [59]:

viewer.add_image(the_difference, opacity=0.3, colormap="green")
viewer.add_image(the_difference_2, opacity=0.3, colormap="green")
viewer.add_image(the_difference_3, opacity=0.3, colormap="green")
viewer.add_image(the_difference_4, opacity=0.3, colormap="green")
viewer.add_image(the_difference_5, opacity=0.3, colormap="green")
viewer.add_image(the_difference_6, opacity=0.3, colormap="green")
viewer.add_image(the_difference_7, opacity=0.3, colormap="green")
viewer.add_image(the_difference_8, opacity=0.3, colormap="green")
viewer.add_image(the_difference_9, opacity=0.3, colormap="green")
viewer.add_image(the_difference_10, opacity=0.3, colormap="green")
viewer.add_image(the_difference_11, opacity=0.3, colormap="green")
viewer.add_image(the_difference_12, opacity=0.3, colormap="green")
viewer.add_image(the_difference_13, opacity=0.3, colormap="green")
viewer.add_image(the_difference_14, opacity=0.3, colormap="green")
viewer.add_image(the_difference_15, opacity=0.3, colormap="green")
viewer.add_image(the_difference_16, opacity=0.3, colormap="green")

<Image layer 'the_difference_16' at 0x19c12ec95a0>

In [104]:
img = obj_four.astype(int)
unique_IDs = np.unique(img)
output_seg = np.zeros_like(img)
for ID in unique_IDs:
    seeds = np.zeros_like(img)
    if ID == 0:
        pass
    else:
        obj = img==ID
        size = np.count_nonzero(obj)
        count = 0
        print(f"Working on ID # {ID}")
        while size != 0:
            print(np.count_nonzero(obj))
            erode = skimage.morphology.isotropic_erosion(obj, 0.1, spacing=scale, )
            filled_remainder = skimage.segmentation.watershed(~obj, markers=erode, mask=obj)
            centers = label(np.logical_xor(obj, filled_remainder))
            
            count = np.max(seeds)+1
            centers = skimage.segmentation.relabel_sequential(centers, count)[0]

            seeds = np.add(centers, seeds)

            obj = erode
            size = np.count_nonzero(obj)

        watershed = skimage.segmentation.watershed(~img==ID, markers=seeds, mask=img==ID)
        relabeling = np.max(output_seg)+1
        watershed = skimage.segmentation.relabel_sequential(watershed, relabeling)[0]
        output_seg = np.add(watershed, output_seg)

        print(f"Completed ID # {ID}")

### TO-DO: add in some additional connectivity stuff; hoping to reduce the number of seeds being extracted.

Working on ID # 4
26239
19979
14712
10665
7616
5368
3725
2542
1650
1031
632
349
160
65
16
2
Completed ID # 4


In [106]:
watershed_test = skimage.segmentation.watershed(~img==4, markers=seeds, mask=img==4)

In [108]:
np.unique(watershed_test)

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130])

In [107]:
viewer.add_labels(watershed_test)

<Labels layer 'watershed_test' at 0x19ba9d3b640>

In [102]:
np.max(seeds)

130

In [96]:
viewer.add_labels(seeds)

<Labels layer 'output_array' at 0x19c0d699f30>

In [105]:
viewer.add_image(watershed)

<Image layer 'watershed [1]' at 0x19c0dcd6a70>

In [ ]:
img = lyso_basic_labels
output_array = np.zeros_like(img)
for ID in unique_IDs:
    obj = img==ID
    while np.count_nonzero(obj) != 0:
        center = obj
        ph = label(obj)
        ph_lst = np.unique(obj)
        while len(ph_lst) == 2:
            center = obj
            obj = skimage.morphology.isotropic_erosion(obj, 0.1, spacing=scale)
            ph_lst = label(obj)
        while len(ph_lst) > 2:
            for id in ph_lst:
                obj = obj==id
                while np.count_nonzero(obj) != 0:
                    center = obj
                    obj = label(skimage.morphology.isotropic_erosion(obj, 0.1, spacing=scale))
            ph_lst = np.unique(obj)
        output_array = np.add(center*ID, output_array)




SyntaxError: incomplete input (326180696.py, line 5)

In [ ]:
four_eroded = scipy.ndimage.morphology.binary_erosion(obj_four, structure=ball(1), brute_force=False)
four_eroded_brute = scipy.ndimage.morphology.binary_erosion(obj_four, structure=ball(3), brute_force=True)

C:\Users\Shannon\AppData\Local\Temp\ipykernel_22540\3202761172.py:1: DeprecationWarning: Please use `binary_erosion` from the `scipy.ndimage` namespace, the `scipy.ndimage.morphology` namespace is deprecated.
  four_eroded = scipy.ndimage.morphology.binary_erosion(obj_four, structure=ball(1), brute_force=False)
C:\Users\Shannon\AppData\Local\Temp\ipykernel_22540\3202761172.py:2: DeprecationWarning: Please use `binary_erosion` from the `scipy.ndimage` namespace, the `scipy.ndimage.morphology` namespace is deprecated.
  four_eroded_brute = scipy.ndimage.morphology.binary_erosion(obj_four, structure=ball(3), brute_force=True)


In [ ]:
viewer.add_image(four_eroded, opacity=0.3, colormap="bop orange")
viewer.add_image(four_eroded_brute, opacity=0.3, colormap="magenta")

<Image layer 'four_eroded_brute [1]' at 0x28c8ec0db70>

In [ ]:
new_array = np.zeros(lyso_basic_labels.shape, int)
np.unique(new_array), new_array.shape, lyso_basic_labels.shape

(array([0]), (17, 704, 704), (17, 704, 704))

In [ ]:
obj_one_again = obj_one*1
obj_two_again = obj_two*2
np.unique(obj_one_again), np.unique(obj_two_again)

(array([0, 1]), array([0, 2]))

In [ ]:
new_array = np.add(obj_one_again, obj_two_again)
np.unique(new_array)

array([0, 1, 2])

In [ ]:
viewer.add_image(new_array)

<Image layer 'new_array' at 0x20c253b75b0>

In [ ]:
unique_IDs_test = [0, 4] #np.unique(new_array)
unique_IDs_test

[0, 4]

In [ ]:
empty_list = np.zeros_like(lyso_basic_labels)
count = 0
for ID in unique_IDs_test:
    obj = lyso_basic_labels==ID
    while np.count_nonzero(obj) != 0:
        centers = obj
        obj = label(skimage.morphology.binary_erosion(obj))
        count = count+1
        
    empty_list = np.add(centers*ID, empty_list)

    ## This seems to be working some what, but not sure why it is losing some of the objects

KeyboardInterrupt: 

In [ ]:
count

264

In [ ]:
np.count_nonzero(empty_list)

381

In [ ]:
np.unique(empty_list)

array([ 0,  4,  8, 12, 16, 20, 24, 28, 32, 36])

In [ ]:
viewer.add_image(lyso_basic_labels==4)
viewer.add_image(empty_list)

<Image layer 'empty_list' at 0x20c8c9b5e70>

In [ ]:
np.shape(empty_list), np.shape(obj_one)

((17, 704, 704), (17, 704, 704))

In [ ]:
np.unique(empty_list), np.unique(obj_one_again)

(array([0, 1]), array([0, 1]))

In [ ]:
np.all(empty_list == obj_one_again)

False

In [ ]:
eroded = topology_preserving_thinning(cleaned_img2, 1, 10)

In [ ]:
viewer.add_image(eroded, colormap="red", opacity=0.3)

In [ ]:
top_hat = skimage.morphology.white_tophat(cleaned_img2)
black_top_hat = skimage.morphology.black_tophat(cleaned_img2)

In [ ]:
viewer.add_image(top_hat, colormap="yellow", opacity=0.3)
viewer.add_image(black_top_hat, colormap="bop orange", opacity=0.3)

In [ ]:
lyso_watershed_labels = watershed(struct_img, mask=cleaned_img2)

In [ ]:
viewer.add_image(lyso_watershed_labels, colormap="green")

In [ ]:
struct_img.shape

In [ ]:
local_maxima = skimage.morphology.local_maxima_maxima(struct_img, selem=np.ones((3,3,3)))

In [ ]:
viewer.add_image(local_maxima)

## LABELING prototype

In [ ]:
lysosome_labels = label_uint16(cleaned_img2)

In [ ]:
lysosome_labels.dtype

## Visualize with `napari` 1
Visualize the first-pass segmentation and labeling with `napari`.

In [ ]:
viewer = napari.Viewer()

In [ ]:
viewer.add_image(
    struct_img)    

viewer.add_image(
    cleaned_img2,
    opacity=0.3,)

viewer.add_labels(
    lysosome_labels,
    opacity=0.3)


## SAVE inferred lysosomes to .tif file

In [ ]:
out_file_n = export_inferred_organelle(lysosome_labels, "lyso", meta_dict, out_data_path)

----------
## DEFINE `_infer_lysosomes` function

Based on the _prototyping_ above define the function to infer nuclei. 

> NOTE: these functions mainly serve for downstream prototyping in the notebooks. Each step above has an independent function that is implemented in the plugin for easy of use.

In [ ]:
##########################
#  infer_LYSOSOMES
##########################
def _infer_lyso(
                                in_img: np.ndarray,
                                lyso_ch: int,
                                median_sz: int,
                                gauss_sig: float,
                                dot_scale_1: float,
                                dot_cut_1: float,
                                dot_scale_2: float,
                                dot_cut_2: float,
                                dot_scale_3: float,
                                dot_cut_3: float,
                                dot_method: str,
                                fil_scale_1: float,
                                fil_cut_1: float,
                                fil_scale_2: float, 
                                fil_cut_2: float, 
                                fil_scale_3: float, 
                                fil_cut_3: float,
                                fil_method: str,
                                min_hole_w: int,
                                max_hole_w: int,
                                small_obj_w: int,
                                fill_filter_method: str
                            ) -> np.ndarray:
    """
    Procedure to infer lysosome from linearly unmixed input,
    
    Parameters
    ------------
    in_img: 
        a 3d image containing all the channels
    median_sz: 
        width of median filter for signal
    gauss_sig: 
        sigma for gaussian smoothing of  signal
    dot_scale: 
        scales (log_sigma) for dot filter (1,2, and 3)
    dot_cut: 
        threshold for dot filter thresholds (1,2,and 3)
    fil_scale: 
        scale (log_sigma) for filament filter
    fil_cut: 
        threshold for filament fitered threshold
    min_hole_w: 
        hole filling min for nuclei post-processing
    max_hole_w: 
        hole filling cutoff for nuclei post-processing
    small_obj_w: 
        minimu object size cutoff for nuclei post-processing
    fill_filter_method:
        to fill snall holes and remove small objects in "3D" or "slice-by-slice"

    Returns
    -------------
    lyso_object
        mask defined extent of lysosome object

    """
    ###################
    # EXTRACT
    ###################    
    lyso = select_channel_from_raw(in_img, lyso_ch)

     ###################
    # PRE_PROCESSING
    ###################    
    lyso1 =  scale_and_smooth(lyso,
                             median_size = median_sz, 
                             gauss_sigma = gauss_sig)
   ###################
    # CORE_PROCESSING
    ###################
    bw_dot = dot_filter_3(lyso1, dot_scale_1, dot_cut_1, dot_scale_2, dot_cut_2, dot_scale_3, dot_cut_3, dot_method)

    bw_filament = filament_filter_3(lyso1, fil_scale_1, fil_cut_1, fil_scale_2, fil_cut_2, fil_scale_3, fil_cut_3, fil_method)

    bw = np.logical_or(bw_dot, bw_filament)

    ###################
    # POST_PROCESSING
    ###################
    struct_obj = fill_and_filter_linear_size(bw, hole_min=min_hole_w, hole_max=max_hole_w, min_size=small_obj_w, method=fill_filter_method)

    ###################
    # LABELING
    ###################
    struct_obj1 = label_uint16(struct_obj)

    return struct_obj1


## DEFINE `_fixed_infer_lyso` function


In [ ]:
##########################
#  fixed_infer_nuclei
##########################
def _fixed_infer_lyso(in_img: np.ndarray) -> np.ndarray:
    """
    Procedure to infer lysosome from linearly unmixed input with *fixed parameters*
    Parameters
    ------------
    in_img: 
        a 3d image containing all the channels

    Returns
    -------------
    lyso_object
        mask defined extent of NU
    """
    lyso_ch = 1
    median_sz = 3
    gauss_sig = 1.34
    dot_scale_1 = 5
    dot_cut_1 = 0.09
    dot_scale_2 = 2.5
    dot_cut_2 = 0.07
    dot_scale_3 = 1
    dot_cut_3 = 0.01
    dot_method = "3D"
    fil_scale_1 = 1
    fil_cut_1 = 0.15
    fil_scale_2 = 0
    fil_cut_2 = 0
    fil_scale_3 = 0
    fil_cut_3 = 0
    fil_method = "3D"
    min_hole_w = 0
    max_hole_w = 25
    small_obj_w = 0
    method = "3D"

    return _infer_lyso(  
        in_img,
        lyso_ch,
        median_sz,
        gauss_sig,
        dot_scale_1,
        dot_cut_1,
        dot_scale_2,
        dot_cut_2,
        dot_scale_3,
        dot_cut_3,
        dot_method,
        fil_scale_1,
        fil_cut_1,
        fil_scale_2,
        fil_cut_2,
        fil_scale_3,
        fil_cut_3,
        fil_method,
        min_hole_w,
        max_hole_w,
        small_obj_w,
        method)

# TEST `_infer_lyso` function defined above

In [ ]:
_lyso_object =  _fixed_infer_lyso(img_data) 

_lyso_object.dtype

In [ ]:
np.all(lysosome_labels == _lyso_object)

---------------------
# TEST `infer_lyso` exported functions

The prototype `_infer_lyso` was copied to the [`.organelles.lysosome`](../infer_subc/organelles/lysosome.py) sub-module.

In [ ]:
from infer_subc.organelles.lysosome import fixed_infer_lyso

lyso_object =  fixed_infer_lyso(img_data)

In [ ]:
np.all(_lyso_object == lyso_object)

## Visualize  2


In [ ]:
viewer.add_image(
    _lyso_object,
    opacity=0.3)

viewer.add_labels(
    lyso_object,
    opacity=0.3)

In [ ]:
from napari.utils.notebook_display import nbscreenshot

nbscreenshot(viewer, canvas_only=True)

In [ ]:
viewer.close()

-------------
## SUMMARY

The above details how the lysosome object is inferred.  

### NEXT: INFER MITOCHONDRIA

proceed to [05_infer_mito.ipynb](./05_infer_mito.ipynb)
